## 0. Install and Import Dependencies

In [1]:
# !pip install ibm_watson
# !pip install python-ffmpeg
# !pip install ffmpeg moviepy

In [2]:
import subprocess
import moviepy.editor as mp
from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

## 1. Extract Audio

In [3]:
# Here we are calling the moviepy library to convert the video to mkv
my_clip = mp.VideoFileClip(r"video1.mp4")
my_clip

In [4]:
my_clip.audio.write_audiofile(r"my_result.wav")

MoviePy - Writing audio in my_result.wav


MoviePy - Done.


## 2. Setup STT Service

In [2]:
# Navigate to https://cloud.ibm.com/catalog?category=ai#servicesv ---->speech to text

apikey = ''
url = ''

In [6]:
# Setup service
authenticator = IAMAuthenticator(apikey)
stt = SpeechToTextV1(authenticator=authenticator)
stt.set_service_url(url)

In [27]:
#!pip install ffmpeg

In [28]:
#!pip list

# 3. Compress and Split Audio

In [7]:
import subprocess
import os
command = 'ffmpeg -i my_result.wav -vn -ar 44100 -ac 2 -b:a 192k my_result.mp3'
#subprocess.Popen('echo %s' %command)
subprocess.call(command, shell=True)

0

In [8]:
command = 'ffmpeg -i my_result.mp3 -f segment -segment_time 360 -c copy %03d.mp3'
subprocess.call(command, shell=True)

0

In [10]:
files = []
for filename in os.listdir('.'):
    if filename.endswith(".mp3") and filename !='my_result.mp3':
        files.append(filename)
files.sort()

In [11]:
files


['000.mp3', '001.mp3', '002.mp3']

# 4. Perform Conversion

In [13]:
results = []
for filename in files:
    with open(filename, 'rb') as f:
        res = stt.recognize(audio=f, content_type='audio/mp3', model='en-AU_NarrowbandModel', continuous=True, \
                           inactivity_timeout=360).get_result()
        results.append(res)

In [14]:
text = []
for file in results:
    for result in file['results']:
        text.append(result['alternatives'][0]['transcript'].rstrip() + '.\n')

In [ ]:
text

# 5. Save to output file

In [ ]:
with open('output.txt', 'w') as out:
    out.writelines(text)